In [254]:
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import networkx as nx

import matplotlib.pyplot as plt

In [255]:
!python3 -m spacy download en_core_web_sm

     ---------------------------------------- 12.8/12.8 MB 9.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


You should consider upgrading via the 'C:\Users\T-GAMER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [256]:
book=[]

In [257]:
#carregando modelo
model = 'en_core_web_sm'
NER = spacy.load(model)


In [258]:
import os
base_path = r'C:\Users\T-GAMER\Desktop\FINAL_CR'
print(os.listdir(base_path))

['Book', 'Data', 'EventosChave.txt', 'Notebooks']


In [259]:
import os
book_path = os.path.join(base_path, 'Book')
books = [b for b in os.scandir(book_path) if 'txt' in b.name]

In [260]:
books[8]

<DirEntry 'percy_jackson_book_9.txt'>

In [236]:
#mudar o book[x] sendo x  o livro o qual voce quer
book_text= open(books[9], encoding='ISO-8859-1').read()
NER.max_length = 2000000
book_1 = NER(book_text)

In [237]:
#-----------------------------------
#displacy.render(book_1[0:2000], style="ent", jupyter=True)

In [238]:
sentence= []

for i in book_1.sents:
    ent_list=(ent.text for ent in i.ents)
    sentence.append({"sentence":i,'entity':ent_list})

In [239]:
sentence=pd.DataFrame(sentence)
sentence.head()

,sentence,entity
0,"( , Contents, \n\n, I, :, Ja...",<generator object <genexpr> at 0x00000209C1C74...
1,"(Leo, \n\n, XXXIV, :, Leo, \n\n, XXXV, :, Leo,...",<generator object <genexpr> at 0x00000209C1C74...
2,"(Reyna, \n\n, XXXIX, :, Reyna, \n\n, XL, :, Re...",<generator object <genexpr> at 0x00000209C1C78...
3,"(Nico, \n\n, LIV, :, Nico, \n\n, LV, :, Nico, ...",<generator object <genexpr> at 0x00000209C1C78...
4,"(Leo, \n\n, Glossary, \n , Rick, Riordan, is, ...",<generator object <genexpr> at 0x00000209C1C78...


In [240]:
character_df=pd.read_excel('../Data/Character_ALL.xlsx')

In [241]:
character_df["first_name"] = character_df["Character"].apply( lambda x: x.split(" ",1)[0])

In [242]:

def filter_out(entity_col, character_df):
    return [x for x in entity_col if x in list(character_df['first_name'])]


sentence['character_entity'] = sentence['entity'].apply(lambda x: filter_out(x, character_df))

In [243]:
df_filter = sentence[sentence['character_entity'].apply(lambda x: len(x) > 0)].reset_index()

In [244]:
df_filter.head()

,index,sentence,entity,character_entity
0,32,"(Piper, and, Annabeth, were, disguised, as, lo...",<generator object <genexpr> at 0x00000209C1D32...,[Annabeth]
1,36,"(She, \n, resembled, an, ancient, statue, of, ...",<generator object <genexpr> at 0x00000209C1D3D...,"[Aphrodite, Jason]"
2,38,"(Dating, a, girl, whose, mom, was, the, goddes...",<generator object <genexpr> at 0x00000209C1D3D...,[Jason]
3,39,"(Jason, glanced, uphill, .)",<generator object <genexpr> at 0x00000209C1D01...,[Jason]
4,47,"(Knowing, her, hatred, of, spiders, ,, Jason, ...",<generator object <genexpr> at 0x00000209C1D10...,[Jason]


In [245]:

window_size = 5
char_list = []
for window in df_filter['character_entity'].rolling(window=window_size):
    if window.size == window_size:
        char_list.extend(sum(window.dropna(), [])) 

In [246]:

unique_list = []

prev_element = None


for element in char_list:
    if element != prev_element:
        unique_list.append(element)
 
    prev_element = element

In [248]:

source = []
target = []

for i in range(len(unique_list)-1):
    source.append(unique_list[i])
    target.append(unique_list[i+1])       

relationship_df = pd.DataFrame({"Source" : source,
                               "Target" : target})

In [249]:
relationship_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)
relationship_df

,Source,Target
0,Annabeth,Aphrodite
1,Aphrodite,Jason
2,Aphrodite,Jason
3,Aphrodite,Jason
4,Annabeth,Jason
...,...,...
9146,Juno,Jupiter
9147,Juno,Venus
9148,Jupiter,Venus
9149,Atlas,Jupiter


In [250]:

relationship_df["Weight"] = 1
relationship_df = relationship_df.groupby(["Source","Target"], sort=False, as_index=False).sum()

In [251]:
relationship_df=relationship_df.sort_values(by='Weight',ascending=False)


In [253]:
relationship_df.to_csv("../Data/relationship_percy_10", index=False) 